In [13]:
import face_recognition
import os
import cv2
import numpy as np
import csv
from datetime import datetime
from tkinter import *
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

# Directory containing known faces
known_faces_dir = 'known_faces'
known_face_encodings = []
known_face_names = []

# Load known faces
for person_name in os.listdir(known_faces_dir):
    person_dir = os.path.join(known_faces_dir, person_name)
    for image_name in os.listdir(person_dir):
        image_path = os.path.join(person_dir, image_name)
        image = face_recognition.load_image_file(image_path)
        encoding = face_recognition.face_encodings(image)[0]
        known_face_encodings.append(encoding)
        known_face_names.append(person_name)

# Prepare the attendance CSV file
attendance_file = 'attendance.csv'
header = ['Name', 'Date']
date_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Check if the CSV file exists, if not create it and write the header
if not os.path.exists(attendance_file):
    with open(attendance_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)

# GUI application
class FaceRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Face Recognition Attendance System")

        self.label = Label(root, text="Face Recognition Attendance System", font=("Arial", 16))
        self.label.pack(pady=20)

        self.load_btn = Button(root, text="Load Group Photo", command=self.load_image)
        self.load_btn.pack(pady=10)

        self.view_attendance_btn = Button(root, text="View Attendance", command=self.view_attendance)
        self.view_attendance_btn.pack(pady=10)

        self.image_label = Label(root)
        self.image_label.pack(pady=20)

    def load_image(self):
        self.filepath = filedialog.askopenfilename(
            filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")]
        )
        if self.filepath:
            self.process_image()

    def process_image(self):
        # Load the group photo
        group_image = face_recognition.load_image_file(self.filepath)

        # Find all faces and face encodings in the group photo
        face_locations = face_recognition.face_locations(group_image)
        face_encodings = face_recognition.face_encodings(group_image, face_locations)

        # Convert to OpenCV format
        group_image_cv2 = cv2.cvtColor(group_image, cv2.COLOR_RGB2BGR)

        # List to keep track of marked names to avoid duplicates
        marked_names = []

        # Loop through faces in the group photo
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Mark attendance if the person is identified and not yet marked
            if name != "Unknown" and name not in marked_names:
                with open(attendance_file, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([name, date_str])
                    marked_names.append(name)

            # Draw a rectangle around the face
            cv2.rectangle(group_image_cv2, (left, top), (right, bottom), (0, 0, 255), 2)
            # Draw a label with a name below the face
            cv2.rectangle(group_image_cv2, (left, bottom), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(group_image_cv2, name, (left, bottom + 15), font, 2, (255, 255, 255), 3)

        # Resize the image to fit the window
        max_size = 800  # Maximum size of the displayed image
        height, width = group_image_cv2.shape[:2]
        scaling_factor = max_size / max(height, width)
        if scaling_factor < 1:
            group_image_cv2 = cv2.resize(group_image_cv2, (int(width * scaling_factor), int(height * scaling_factor)))

        # Convert back to PIL format
        group_image_rgb = cv2.cvtColor(group_image_cv2, cv2.COLOR_BGR2RGB)
        group_image_pil = Image.fromarray(group_image_rgb)
        group_image_tk = ImageTk.PhotoImage(image=group_image_pil)

        # Update the image label
        self.image_label.config(image=group_image_tk)
        self.image_label.image = group_image_tk

        # Save the result (optional)
        output_path = 'identified_faces.jpg'
        cv2.imwrite(output_path, group_image_cv2)
        messagebox.showinfo("Success", f"Image processed and saved to {output_path}")

    def view_attendance(self):
        if os.path.exists(attendance_file):
            os.startfile(attendance_file)  # This will open the CSV file using the default application
        else:
            messagebox.showwarning("No Data", "No attendance data found.")

if __name__ == "__main__":
    root = Tk()
    app = FaceRecognitionApp(root)
    root.mainloop()
